In [1]:
import matplotlib.pyplot as plt # type: ignore
from collections import Counter
import numpy as np # type: ignore
import aes

In [8]:
def hamming_weight(n):
    count = 0
    while n:
        count += n & 1
        n >>= 1
    return count

hamming_weights = np.array([hamming_weight(i) for i in range(256)])

In [3]:
def get_inputs():
    with open("anthony/data/dpa-aes/aes_data.txt", 'r') as file:
        lines = file.readlines()

    input_hex = []
    output_hex = []
    for line in lines:
        hex_string = ''.join(line.split())  # Remove any spaces in the line
        input_hex.append(hex_string[:32])  # First 32 characters for input
        output_hex.append(hex_string[32:])  # Remaining characters for outpu

    return input_hex, output_hex

def load_traces():
    traces = []
    with open("anthony/data/dpa-aes/aes_samples.txt", 'r') as f:
        for line in f:
            # Replace commas with dots and split the line by tabs
            trace = list(map(float, line.strip().replace(',', '.').split('\t')))
            traces.append(trace)
    return traces

In [10]:
inputs, outputs = get_inputs()
input_split = np.array([[row[i:i+2] for i in range(0, len(row), 2)] for row in inputs])
decimal_array = np.array([[int(hex_string, 16) for hex_string in row] for row in input_split])
N = decimal_array.shape[0]
sbox_array = np.empty_like(decimal_array)
traces = np.array(load_traces())
SBOX = np.array(aes.AES_Sbox)
max_values = np.empty((16, 2**8))
sbox_0 = []

for i in range(1):
    print(f"KEY: {i}")
    key_added = decimal_array ^ i
    sbox_output = SBOX[key_added]
    results = []

    # Take columns of hamming weight (Each SBOX)
    for column in range(sbox_output.shape[1]):
        sbox_output_column = sbox_output[:, column][:, np.newaxis]
        result = sbox_output_column * traces
        row_sum = np.sum(result, axis=0)
        results.append(row_sum)

    # ABOVE TO DIVISION
    Wi_Hi = np.array(results) # Multiplicar todo esto por N (valor por valor)
    Hi = np.array(np.sum(sbox_output, axis=0), dtype=np.int64)[:, np.newaxis]
    Wi = np.array([np.sum(traces, axis=0)])
    argFirst = Wi_Hi * N

    argSecond = Hi * Wi
    divisionAbove = argFirst - argSecond

    # BELOW DIVISION
    squared_traces = traces ** 2
    squared_hamming = sbox_output ** 2
    Wi2 =  (np.array([np.sum(squared_traces, axis=0)])) * N
    Wi_2 = np.square(Wi)
    sqrootFirst = np.sqrt((Wi2-Wi_2))
    Hi2 =  (np.array([np.sum(squared_hamming, axis=0)], dtype=np.int64)).reshape(-1,1) * N
    Hi_2 = Hi ** 2
    sqrootSecond = np.sqrt((Hi2-Hi_2))
    divisionBelow = sqrootFirst * sqrootSecond

    final_result = np.divide(divisionAbove, divisionBelow)
    sbox_0.append(final_result[0])
    max_abs = np.array(np.max(np.abs(final_result), axis=1)).reshape(-1,1)
    max_values[:,i] = max_abs.flatten()

key = np.argmax(max_values, axis=1)
key_hex = [format(x, 'x') for x in key]
final_key = [x.zfill(2).upper() for x in key_hex]

print(f"FINAL KEY: {final_key}")

KEY: 0
KEY: 1
KEY: 2
KEY: 3
KEY: 4
KEY: 5
KEY: 6
KEY: 7
KEY: 8
KEY: 9
KEY: 10
KEY: 11
KEY: 12
KEY: 13
KEY: 14
KEY: 15
KEY: 16
KEY: 17
KEY: 18
KEY: 19
KEY: 20
KEY: 21
KEY: 22
KEY: 23
KEY: 24
KEY: 25
KEY: 26
KEY: 27
KEY: 28
KEY: 29
KEY: 30
KEY: 31
KEY: 32
KEY: 33
KEY: 34
KEY: 35
KEY: 36
KEY: 37
KEY: 38
KEY: 39
KEY: 40
KEY: 41
KEY: 42
KEY: 43
KEY: 44
KEY: 45
KEY: 46
KEY: 47
KEY: 48
KEY: 49
KEY: 50
KEY: 51
KEY: 52
KEY: 53
KEY: 54
KEY: 55
KEY: 56
KEY: 57
KEY: 58
KEY: 59
KEY: 60
KEY: 61
KEY: 62
KEY: 63
KEY: 64
KEY: 65
KEY: 66
KEY: 67
KEY: 68
KEY: 69
KEY: 70
KEY: 71
KEY: 72
KEY: 73
KEY: 74
KEY: 75
KEY: 76
KEY: 77
KEY: 78
KEY: 79
KEY: 80
KEY: 81
KEY: 82
KEY: 83
KEY: 84
KEY: 85
KEY: 86
KEY: 87
KEY: 88
KEY: 89
KEY: 90
KEY: 91
KEY: 92
KEY: 93
KEY: 94
KEY: 95
KEY: 96
KEY: 97
KEY: 98
KEY: 99
KEY: 100
KEY: 101
KEY: 102
KEY: 103
KEY: 104
KEY: 105
KEY: 106
KEY: 107
KEY: 108
KEY: 109
KEY: 110
KEY: 111
KEY: 112
KEY: 113
KEY: 114
KEY: 115
KEY: 116
KEY: 117
KEY: 118
KEY: 119
KEY: 120
KEY: 121
KEY: 122
KEY

In [9]:
hamming_weights[sbox_output_column]

array([[4],
       [3],
       [3],
       ...,
       [4],
       [6],
       [4]])